# Task 1
Исследуйте способы композиции из module-sklearn.ensemble.

Подготовьте данные, уберите аномалии. Разбейте датасет, проведите обучение следующих композиций:

Бэггинг
Бустинг
Стекинг
В качестве базовых алгоритмов можете выбирать любые из рассмотренных. Проведите работу над поиском гиперпараметров для композиций и базовых алгоритмов.

Сравните качество работы и скорость работы базовых алгоритмов и композиций. Свои действия снабжайте пояснениями. Не пренебрегайте оформлением нотбуков, придерживайтесь структурированного изложения результатов своего исследования.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR

In [2]:
forest_fires = pd.read_csv("../data/forest_fires_preprocessed.csv")
forest_fires.pop('Unnamed: 0')
forest_fires.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


##### Данных мало для выявления аномалий


In [3]:
for x in ['X', 'Y', 'month', 'day',
          'FFMC', 'DMC', 'DC', 'ISI',
          'temp', 'RH', 'wind', 'rain']:
    q75,q25 = np.percentile(forest_fires.loc[:,x],[75,25])
    intr_qr = q75-q25

    max = q75+(1.5*intr_qr)
    min = q25-(1.5*intr_qr)

    forest_fires.loc[forest_fires[x] < min,x] = np.nan
    forest_fires.loc[forest_fires[x] > max,x] = np.nan

In [4]:
forest_fires.isnull().sum()

X         0
Y        51
month    76
day       0
FFMC     53
DMC      17
DC        8
ISI      23
temp      2
RH       12
wind     13
rain      8
area      0
dtype: int64

##### Продолжение

In [5]:
forest_fires = pd.read_csv("../data/forest_fires_preprocessed.csv")
forest_fires.pop('Unnamed: 0')
forest_fires.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [6]:
X = forest_fires.drop(['area'], 1)
y = forest_fires['area']

In [7]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [8]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
def print_metrics(metrics, names = ["R2", "MSE", "RMSE", "MAE"]):
    for i, m in enumerate(metrics):
        print(f'{names[i]} metric\nmean: {-mean(m)}\nstd: {std(m)}\n')

#### Бэггинг


In [10]:
BagReg = BaggingRegressor()
BagReg.fit(X_train, y_train)
ybr_pred = BagReg.predict(X_test)

In [11]:
scores = list()

scores.append(cross_val_score(BagReg, X, y, cv=3, scoring='r2'))
scores.append(cross_val_score(BagReg, X, y, cv=3, scoring='neg_mean_squared_error'))
scores.append(cross_val_score(BagReg, X, y, cv=3, scoring='neg_root_mean_squared_error'))
scores.append(cross_val_score(BagReg, X, y, cv=3, scoring='neg_mean_absolute_error'))

In [12]:
print_metrics(scores)

R2 metric
mean: 1726.4675080492198
std: 2441.49809399025

MSE metric
mean: 4887.062141456229
std: 2617.2941999960885

RMSE metric
mean: 64.47442687440133
std: 22.32741285548147

MAE metric
mean: 29.99807181707517
std: 6.543662077685034



In [13]:
print('R2:', metrics.r2_score(y_test, ybr_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, ybr_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, ybr_pred))
print('Root Mean Squared Error:', (metrics.mean_squared_error(y_test, ybr_pred, squared=False)))

R2: -0.35359618634740775
Mean Absolute Error: 18.24204487179487
Mean Squared Error: 1464.743334838319
Root Mean Squared Error: 38.2719653903261


In [15]:
forest_fires_preds = pd.DataFrame({'Actual': y_test, 'Predicted': ybr_pred})
forest_fires_preds

,Actual,Predicted
209,23.41,34.894
397,3.63,13.110
202,13.05,13.469
214,28.66,17.196
24,0.00,1.162
...,...,...
288,0.00,1.881
71,0.00,3.172
302,0.00,0.840
433,15.34,5.039


##### Бустинг

In [17]:
GrBReg = GradientBoostingRegressor()
GrBReg.fit(X_train, y_train)
yGrBReg_pred = GrBReg.predict(X_test)

In [18]:
scores2 = list()

scores2.append(cross_val_score(GrBReg, X, y, cv=3, scoring='r2'))
scores2.append(cross_val_score(GrBReg, X, y, cv=3, scoring='neg_mean_squared_error'))
scores2.append(cross_val_score(GrBReg, X, y, cv=3, scoring='neg_root_mean_squared_error'))
scores2.append(cross_val_score(GrBReg, X, y, cv=3, scoring='neg_mean_absolute_error'))

In [19]:
print_metrics(scores2)

R2 metric
mean: 1202.1568360396723
std: 1700.021108580321

MSE metric
mean: 4766.080144382992
std: 2769.540576230808

RMSE metric
mean: 66.19691408700156
std: 20.103215459992576

MAE metric
mean: 25.379780920300618
std: 2.9166113532227396



In [20]:
print('R2:', metrics.r2_score(y_test, yGrBReg_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, yGrBReg_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, yGrBReg_pred))
print('Root Mean Squared Error:', (metrics.mean_squared_error(y_test, yGrBReg_pred,squared=False)))

R2: -1.2701187107387275
Mean Absolute Error: 21.83454713605541
Mean Squared Error: 2456.5238025817644
Root Mean Squared Error: 49.56333122966781


In [21]:
forest_fires_preds = pd.DataFrame({'Actual': y_test, 'Predicted': yGrBReg_pred})
forest_fires_preds

,Actual,Predicted
209,23.41,7.077570
397,3.63,8.987911
202,13.05,6.147696
214,28.66,6.428241
24,0.00,4.718827
...,...,...
288,0.00,7.197325
71,0.00,0.771376
302,0.00,4.558774
433,15.34,4.299133


##### Стекинг

In [22]:
from sklearn.linear_model import ElasticNet

In [23]:
estimators = [ ('lr', RidgeCV()), ('svr', LinearSVR(random_state=0)) ]

In [24]:
StReg = StackingRegressor(estimators=estimators, final_estimator=RandomForestRegressor(n_estimators=13, random_state=0))

In [25]:
StReg.fit(X_train, y_train)

StackingRegressor(estimators=[('lr', RidgeCV(alphas=array([ 0.1,  1. , 10. ]))),
                              ('svr', LinearSVR(random_state=0))],
                  final_estimator=RandomForestRegressor(n_estimators=13,
                                                        random_state=0))

In [26]:
yStReg_pred = StReg.predict(X_test)

In [27]:
scores3 = list()

scores3.append(cross_val_score(StReg, X, y, cv=3, scoring='r2'))
scores3.append(cross_val_score(StReg, X, y, cv=3, scoring='neg_mean_squared_error'))
scores3.append(cross_val_score(StReg, X, y, cv=3, scoring='neg_root_mean_squared_error'))
scores3.append(cross_val_score(StReg, X, y, cv=3, scoring='neg_mean_absolute_error'))

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

In [28]:
print_metrics(scores3)

R2 metric
mean: 298.6770801730409
std: 422.254139238734

MSE metric
mean: 4531.231038495748
std: 3412.7645077365514

RMSE metric
mean: 60.194349283679
std: 30.13090361755189

MAE metric
mean: 20.205026184477052
std: 5.348812958874072



In [29]:
print('R2:', metrics.r2_score(y_test, yStReg_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, yStReg_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, yStReg_pred))
print('Root Mean Squared Error:', (metrics.mean_squared_error(y_test, yStReg_pred,squared=False)))

R2: -1.9098952927134496
Mean Absolute Error: 24.80003944773176
Mean Squared Error: 3148.834030465786
Root Mean Squared Error: 56.114472558028964


In [30]:
forest_fires_preds = pd.DataFrame({'Actual': y_test, 'Predicted': yStReg_pred})
forest_fires_preds

,Actual,Predicted
209,23.41,28.652308
397,3.63,0.194615
202,13.05,3.726923
214,28.66,5.490000
24,0.00,0.000000
...,...,...
288,0.00,16.488462
71,0.00,11.713846
302,0.00,1.390769
433,15.34,23.383846


# Task 2

In [31]:
!pip install catboost

In [32]:
from catboost import CatBoostRegressor
CatBReg = CatBoostRegressor(iterations = 300, loss_function='MAE', task_type = 'CPU')

In [33]:
CatBReg.fit(X_train, y_train)

0:	learn: 13.5372026	total: 171ms	remaining: 51s
1:	learn: 13.5276873	total: 198ms	remaining: 29.5s
2:	learn: 13.5098801	total: 221ms	remaining: 21.9s
3:	learn: 13.5003580	total: 262ms	remaining: 19.4s
4:	learn: 13.4931718	total: 275ms	remaining: 16.2s
5:	learn: 13.4755099	total: 285ms	remaining: 14s
6:	learn: 13.4550193	total: 292ms	remaining: 12.2s
7:	learn: 13.4412493	total: 309ms	remaining: 11.3s
8:	learn: 13.4237118	total: 316ms	remaining: 10.2s
9:	learn: 13.4115808	total: 324ms	remaining: 9.41s
10:	learn: 13.3967951	total: 330ms	remaining: 8.67s
11:	learn: 13.3808957	total: 338ms	remaining: 8.1s
12:	learn: 13.3677122	total: 346ms	remaining: 7.65s
13:	learn: 13.3558037	total: 354ms	remaining: 7.23s
14:	learn: 13.3435446	total: 359ms	remaining: 6.83s
15:	learn: 13.3287391	total: 368ms	remaining: 6.53s
16:	learn: 13.3195311	total: 374ms	remaining: 6.22s
17:	learn: 13.3047417	total: 382ms	remaining: 5.99s
18:	learn: 13.2912173	total: 388ms	remaining: 5.74s
19:	learn: 13.2721797	total

175:	learn: 12.0793957	total: 1.86s	remaining: 1.31s
176:	learn: 12.0722329	total: 1.87s	remaining: 1.3s
177:	learn: 12.0690986	total: 1.87s	remaining: 1.28s
178:	learn: 12.0668407	total: 1.89s	remaining: 1.27s
179:	learn: 12.0450447	total: 1.9s	remaining: 1.27s
180:	learn: 12.0362184	total: 1.91s	remaining: 1.25s
181:	learn: 12.0352580	total: 1.92s	remaining: 1.24s
182:	learn: 12.0322411	total: 1.92s	remaining: 1.23s
183:	learn: 12.0291289	total: 1.93s	remaining: 1.22s
184:	learn: 12.0261681	total: 1.93s	remaining: 1.2s
185:	learn: 12.0241582	total: 1.94s	remaining: 1.19s
186:	learn: 12.0208086	total: 1.95s	remaining: 1.18s
187:	learn: 12.0186026	total: 1.95s	remaining: 1.16s
188:	learn: 12.0158179	total: 1.96s	remaining: 1.15s
189:	learn: 12.0110833	total: 1.97s	remaining: 1.14s
190:	learn: 12.0064934	total: 1.97s	remaining: 1.13s
191:	learn: 12.0048425	total: 1.98s	remaining: 1.11s
192:	learn: 11.9993466	total: 1.99s	remaining: 1.1s
193:	learn: 11.9958379	total: 1.99s	remaining: 1.0

In [34]:
y_cat_pred = CatBReg.predict(X_test)

In [35]:
scoresCat = list()

scoresCat.append(cross_val_score(CatBReg, X, y, cv=3, scoring='r2'))
scoresCat.append(cross_val_score(CatBReg, X, y, cv=3, scoring='neg_mean_squared_error'))
scoresCat.append(cross_val_score(CatBReg, X, y, cv=3, scoring='neg_root_mean_squared_error'))
scoresCat.append(cross_val_score(CatBReg, X, y, cv=3, scoring='neg_mean_absolute_error'))

0:	learn: 18.4711804	total: 6.67ms	remaining: 1.99s
1:	learn: 18.4255912	total: 12.7ms	remaining: 1.89s
2:	learn: 18.4056506	total: 18.6ms	remaining: 1.84s
3:	learn: 18.3687595	total: 23.8ms	remaining: 1.76s
4:	learn: 18.3444640	total: 30ms	remaining: 1.77s
5:	learn: 18.2995523	total: 35.7ms	remaining: 1.75s
6:	learn: 18.2822345	total: 41.3ms	remaining: 1.73s
7:	learn: 18.2599952	total: 46.9ms	remaining: 1.71s
8:	learn: 18.2045109	total: 53ms	remaining: 1.71s
9:	learn: 18.1924257	total: 59.2ms	remaining: 1.72s
10:	learn: 18.1383895	total: 65.9ms	remaining: 1.73s
11:	learn: 18.0981758	total: 72.5ms	remaining: 1.74s
12:	learn: 18.0853847	total: 77.9ms	remaining: 1.72s
13:	learn: 18.0562633	total: 85ms	remaining: 1.74s
14:	learn: 18.0379577	total: 91.6ms	remaining: 1.74s
15:	learn: 18.0067812	total: 98.2ms	remaining: 1.74s
16:	learn: 17.9950547	total: 104ms	remaining: 1.74s
17:	learn: 17.9171029	total: 110ms	remaining: 1.73s
18:	learn: 17.8904345	total: 116ms	remaining: 1.71s
19:	learn: 1

157:	learn: 15.4823254	total: 1.56s	remaining: 1.41s
158:	learn: 15.4622554	total: 1.58s	remaining: 1.4s
159:	learn: 15.4556351	total: 1.59s	remaining: 1.39s
160:	learn: 15.4511725	total: 1.61s	remaining: 1.39s
161:	learn: 15.4489901	total: 1.63s	remaining: 1.38s
162:	learn: 15.4195722	total: 1.64s	remaining: 1.38s
163:	learn: 15.4131735	total: 1.66s	remaining: 1.38s
164:	learn: 15.3844883	total: 1.67s	remaining: 1.37s
165:	learn: 15.3802177	total: 1.68s	remaining: 1.35s
166:	learn: 15.3670561	total: 1.69s	remaining: 1.34s
167:	learn: 15.3625789	total: 1.7s	remaining: 1.34s
168:	learn: 15.3564019	total: 1.71s	remaining: 1.33s
169:	learn: 15.3455497	total: 1.73s	remaining: 1.32s
170:	learn: 15.3400029	total: 1.74s	remaining: 1.31s
171:	learn: 15.3258359	total: 1.75s	remaining: 1.3s
172:	learn: 15.3235979	total: 1.77s	remaining: 1.3s
173:	learn: 15.3131970	total: 1.79s	remaining: 1.29s
174:	learn: 15.3027982	total: 1.81s	remaining: 1.29s
175:	learn: 15.2973875	total: 1.82s	remaining: 1.2

20:	learn: 7.8271517	total: 199ms	remaining: 2.64s
21:	learn: 7.8218981	total: 205ms	remaining: 2.59s
22:	learn: 7.8180430	total: 213ms	remaining: 2.57s
23:	learn: 7.8145920	total: 219ms	remaining: 2.52s
24:	learn: 7.8065571	total: 225ms	remaining: 2.48s
25:	learn: 7.8037358	total: 231ms	remaining: 2.43s
26:	learn: 7.7994158	total: 244ms	remaining: 2.47s
27:	learn: 7.7961654	total: 253ms	remaining: 2.46s
28:	learn: 7.7949181	total: 260ms	remaining: 2.43s
29:	learn: 7.7856277	total: 269ms	remaining: 2.42s
30:	learn: 7.7812765	total: 280ms	remaining: 2.43s
31:	learn: 7.7740778	total: 290ms	remaining: 2.43s
32:	learn: 7.7703382	total: 298ms	remaining: 2.41s
33:	learn: 7.7615582	total: 303ms	remaining: 2.37s
34:	learn: 7.7560117	total: 309ms	remaining: 2.34s
35:	learn: 7.7535986	total: 316ms	remaining: 2.32s
36:	learn: 7.7440861	total: 321ms	remaining: 2.28s
37:	learn: 7.7406239	total: 328ms	remaining: 2.26s
38:	learn: 7.7385236	total: 337ms	remaining: 2.25s
39:	learn: 7.7366505	total: 351

189:	learn: 7.1645996	total: 1.58s	remaining: 915ms
190:	learn: 7.1634013	total: 1.59s	remaining: 907ms
191:	learn: 7.1603133	total: 1.59s	remaining: 897ms
192:	learn: 7.1303105	total: 1.6s	remaining: 887ms
193:	learn: 7.1265793	total: 1.61s	remaining: 879ms
194:	learn: 7.1212541	total: 1.63s	remaining: 876ms
195:	learn: 7.1209548	total: 1.64s	remaining: 871ms
196:	learn: 7.1204130	total: 1.65s	remaining: 864ms
197:	learn: 7.1191905	total: 1.66s	remaining: 855ms
198:	learn: 7.1177672	total: 1.67s	remaining: 847ms
199:	learn: 7.1168152	total: 1.68s	remaining: 838ms
200:	learn: 7.1076397	total: 1.68s	remaining: 828ms
201:	learn: 7.0822859	total: 1.69s	remaining: 818ms
202:	learn: 7.0815293	total: 1.69s	remaining: 808ms
203:	learn: 7.0688548	total: 1.7s	remaining: 799ms
204:	learn: 7.0683756	total: 1.7s	remaining: 790ms
205:	learn: 7.0677695	total: 1.71s	remaining: 780ms
206:	learn: 7.0613427	total: 1.72s	remaining: 773ms
207:	learn: 7.0566905	total: 1.73s	remaining: 767ms
208:	learn: 7.0

56:	learn: 10.8883822	total: 342ms	remaining: 1.46s
57:	learn: 10.8790198	total: 347ms	remaining: 1.45s
58:	learn: 10.8740722	total: 355ms	remaining: 1.45s
59:	learn: 10.8664053	total: 366ms	remaining: 1.46s
60:	learn: 10.8594853	total: 377ms	remaining: 1.48s
61:	learn: 10.8426362	total: 381ms	remaining: 1.46s
62:	learn: 10.8398481	total: 386ms	remaining: 1.45s
63:	learn: 10.8379946	total: 390ms	remaining: 1.44s
64:	learn: 10.8357776	total: 394ms	remaining: 1.43s
65:	learn: 10.8242956	total: 398ms	remaining: 1.41s
66:	learn: 10.8218234	total: 407ms	remaining: 1.42s
67:	learn: 10.8180472	total: 418ms	remaining: 1.43s
68:	learn: 10.8148720	total: 426ms	remaining: 1.42s
69:	learn: 10.8058995	total: 440ms	remaining: 1.45s
70:	learn: 10.8043034	total: 455ms	remaining: 1.47s
71:	learn: 10.8003483	total: 469ms	remaining: 1.48s
72:	learn: 10.7984046	total: 485ms	remaining: 1.51s
73:	learn: 10.7965333	total: 496ms	remaining: 1.51s
74:	learn: 10.7950579	total: 512ms	remaining: 1.53s
75:	learn: 1

...

74:	learn: 16.7021142	total: 612ms	remaining: 1.83s
75:	learn: 16.6980056	total: 621ms	remaining: 1.83s
76:	learn: 16.6929972	total: 630ms	remaining: 1.82s
77:	learn: 16.6794996	total: 645ms	remaining: 1.84s
78:	learn: 16.6734191	total: 663ms	remaining: 1.85s
79:	learn: 16.6469844	total: 672ms	remaining: 1.85s
80:	learn: 16.6415906	total: 678ms	remaining: 1.83s
81:	learn: 16.6364732	total: 682ms	remaining: 1.81s
82:	learn: 16.6158598	total: 687ms	remaining: 1.8s
83:	learn: 16.6108203	total: 692ms	remaining: 1.78s
84:	learn: 16.5964761	total: 697ms	remaining: 1.76s
85:	learn: 16.5866226	total: 701ms	remaining: 1.74s
86:	learn: 16.5658419	total: 705ms	remaining: 1.73s
87:	learn: 16.5354198	total: 710ms	remaining: 1.71s
88:	learn: 16.5250294	total: 714ms	remaining: 1.69s
89:	learn: 16.5088670	total: 719ms	remaining: 1.68s
90:	learn: 16.4830705	total: 724ms	remaining: 1.66s
91:	learn: 16.4564723	total: 728ms	remaining: 1.65s
92:	learn: 16.4437438	total: 733ms	remaining: 1.63s
93:	learn: 16

255:	learn: 14.4356310	total: 1.63s	remaining: 281ms
256:	learn: 14.4299437	total: 1.64s	remaining: 274ms
257:	learn: 14.4213725	total: 1.64s	remaining: 267ms
258:	learn: 14.3956879	total: 1.65s	remaining: 261ms
259:	learn: 14.3805180	total: 1.65s	remaining: 254ms
260:	learn: 14.3781036	total: 1.66s	remaining: 248ms
261:	learn: 14.3650033	total: 1.66s	remaining: 241ms
262:	learn: 14.3510610	total: 1.67s	remaining: 235ms
263:	learn: 14.3183243	total: 1.68s	remaining: 228ms
264:	learn: 14.3130048	total: 1.68s	remaining: 222ms
265:	learn: 14.3101106	total: 1.69s	remaining: 216ms
266:	learn: 14.2850888	total: 1.69s	remaining: 209ms
267:	learn: 14.2813725	total: 1.7s	remaining: 203ms
268:	learn: 14.2778630	total: 1.7s	remaining: 196ms
269:	learn: 14.2772365	total: 1.71s	remaining: 190ms
270:	learn: 14.2731628	total: 1.72s	remaining: 184ms
271:	learn: 14.2619957	total: 1.73s	remaining: 178ms
272:	learn: 14.2590967	total: 1.73s	remaining: 171ms
273:	learn: 14.2511398	total: 1.74s	remaining: 1

125:	learn: 7.4024696	total: 693ms	remaining: 957ms
126:	learn: 7.4006723	total: 699ms	remaining: 952ms
127:	learn: 7.3982980	total: 703ms	remaining: 945ms
128:	learn: 7.3946127	total: 708ms	remaining: 939ms
129:	learn: 7.3893944	total: 715ms	remaining: 935ms
130:	learn: 7.3865015	total: 720ms	remaining: 929ms
131:	learn: 7.3800983	total: 726ms	remaining: 924ms
132:	learn: 7.3785838	total: 730ms	remaining: 917ms
133:	learn: 7.3747576	total: 735ms	remaining: 911ms
134:	learn: 7.3697493	total: 743ms	remaining: 908ms
135:	learn: 7.3632513	total: 748ms	remaining: 902ms
136:	learn: 7.3545110	total: 755ms	remaining: 898ms
137:	learn: 7.3540354	total: 760ms	remaining: 892ms
138:	learn: 7.3525572	total: 765ms	remaining: 886ms
139:	learn: 7.3320535	total: 769ms	remaining: 879ms
140:	learn: 7.3269527	total: 777ms	remaining: 876ms
141:	learn: 7.3259847	total: 783ms	remaining: 871ms
142:	learn: 7.3242912	total: 792ms	remaining: 869ms
143:	learn: 7.3214844	total: 796ms	remaining: 863ms
144:	learn: 

291:	learn: 6.8096054	total: 1.57s	remaining: 43.1ms
292:	learn: 6.8083524	total: 1.58s	remaining: 37.7ms
293:	learn: 6.7956235	total: 1.58s	remaining: 32.3ms
294:	learn: 6.7941507	total: 1.59s	remaining: 26.9ms
295:	learn: 6.7845086	total: 1.59s	remaining: 21.5ms
296:	learn: 6.7831818	total: 1.6s	remaining: 16.2ms
297:	learn: 6.7747694	total: 1.62s	remaining: 10.8ms
298:	learn: 6.7742986	total: 1.63s	remaining: 5.44ms
299:	learn: 6.7505247	total: 1.64s	remaining: 0us
0:	learn: 11.4159056	total: 8.44ms	remaining: 2.52s
1:	learn: 11.3978536	total: 16.2ms	remaining: 2.42s
2:	learn: 11.3809493	total: 35.4ms	remaining: 3.5s
3:	learn: 11.3729402	total: 48.7ms	remaining: 3.6s
4:	learn: 11.3641341	total: 65ms	remaining: 3.83s
5:	learn: 11.3540122	total: 73.5ms	remaining: 3.6s
6:	learn: 11.3367935	total: 85.1ms	remaining: 3.56s
7:	learn: 11.3325458	total: 103ms	remaining: 3.76s
8:	learn: 11.3246330	total: 112ms	remaining: 3.63s
9:	learn: 11.3100896	total: 126ms	remaining: 3.67s
10:	learn: 11.2

156:	learn: 10.3974166	total: 2.35s	remaining: 2.14s
157:	learn: 10.3847860	total: 2.36s	remaining: 2.12s
158:	learn: 10.3803512	total: 2.37s	remaining: 2.1s
159:	learn: 10.3783874	total: 2.38s	remaining: 2.08s
160:	learn: 10.3775064	total: 2.39s	remaining: 2.06s
161:	learn: 10.3719091	total: 2.39s	remaining: 2.04s
162:	learn: 10.3642801	total: 2.4s	remaining: 2.02s
163:	learn: 10.3568142	total: 2.41s	remaining: 2s
164:	learn: 10.3506702	total: 2.42s	remaining: 1.98s
165:	learn: 10.3324234	total: 2.42s	remaining: 1.96s
166:	learn: 10.3093367	total: 2.43s	remaining: 1.94s
167:	learn: 10.3054319	total: 2.44s	remaining: 1.92s
168:	learn: 10.3036530	total: 2.45s	remaining: 1.9s
169:	learn: 10.3019314	total: 2.46s	remaining: 1.89s
170:	learn: 10.3005211	total: 2.47s	remaining: 1.86s
171:	learn: 10.2967369	total: 2.5s	remaining: 1.86s
172:	learn: 10.2950275	total: 2.52s	remaining: 1.85s
173:	learn: 10.2863241	total: 2.53s	remaining: 1.83s
174:	learn: 10.2850313	total: 2.54s	remaining: 1.82s


In [36]:
print_metrics(scoresCat)

R2 metric
mean: 136.2517848642312
std: 192.6225482078852

MSE metric
mean: 4232.885162080135
std: 3296.2443837795613

RMSE metric
mean: 56.65544672033161
std: 31.985082757432426

MAE metric
mean: 16.802829218184286
std: 4.356458869932902



In [37]:
CatBReg.best_score_

{'learn': {'MAE': 11.37640344627963}}

In [38]:
CatBReg.get_best_score()

{'learn': {'MAE': 11.37640344627963}}

In [39]:
print('R2:', metrics.r2_score(y_test, y_cat_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_cat_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_cat_pred))
print('Root Mean Squared Error:', (metrics.mean_squared_error(y_test, y_cat_pred,squared=False)))

R2: -0.1042205670162677
Mean Absolute Error: 12.44036861708362
Mean Squared Error: 1194.8908633474482
Root Mean Squared Error: 34.56719345488506


In [43]:
forest_fires_preds = pd.DataFrame({'Actual': y_test, 'Predicted': y_cat_pred})
forest_fires_preds.iloc[:10]

,Actual,Predicted
209,23.41,0.738552
397,3.63,1.566598
202,13.05,3.109004
214,28.66,0.000358
24,0.00,2.296406
45,0.00,1.835016
9,0.00,1.298585
353,1.72,1.062010
432,0.00,25.056507
13,0.00,1.899060
